In [1]:

import sys
print("--sys.version—")
print(sys.version)


--sys.version—
3.7.11 (default, Jul 27 2021, 09:42:29) [MSC v.1916 64 bit (AMD64)]


In [2]:
import scipy as sp
import numpy as np
import matplotlib as mpl
import pandas as pd
import sklearn as sk
import h5py
import tensorflow as tf
import keras as k
print('scipy ' + sp.__version__)
print('numpy ' + np.__version__)
print('matplotlib ' + mpl.__version__)
print('pandas ' + pd.__version__)
print('sklearn ' + sk.__version__)
print('h5py ' + h5py.__version__)
print('tensorflow ' + tf.__version__)
print('keras ' + k.__version__)

ModuleNotFoundError: No module named 'matplotlib'

In [3]:
import pandas as pd
import sklearn as sk

In [4]:
print('pandas ' + pd.__version__)
print('sklearn ' + sk.__version__)

pandas 0.25.1
sklearn 0.21.2


In [428]:
cgm_raw = pd.read_csv(r'CGMData.csv', index_col = 0)
#cgm_raw.head()
#print(cgm_raw.dtypes)
#cgm_df.info()

In [429]:
cgm_raw['datetime'] = pd.to_datetime(cgm_raw['Date']) + pd.to_timedelta(cgm_raw['Time'])
#cgm_raw.head(3)
#print(cgm_df.dtypes)

In [430]:
cgm_df = cgm_raw[['datetime','Sensor Glucose (mg/dL)']]
#cgm_df.head()

In [431]:
print(cgm_df.dtypes)

datetime                  datetime64[ns]
Sensor Glucose (mg/dL)           float64
dtype: object


In [434]:
insulin_raw = pd.read_csv(r'InsulinData.csv')
insulin_raw['datetime'] = pd.to_datetime(insulin_raw['Date']) + pd.to_timedelta(insulin_raw['Time'])

In [435]:
insulin_df = insulin_raw[['datetime', 'Alarm']]
#insulin_df.head()

In [436]:
print(insulin_df.dtypes)

datetime    datetime64[ns]
Alarm               object
dtype: object


In [437]:
#df=pd.merge(cgm_df, insulin_df, on='DateTime', how='outer')

In [438]:
cgm_df = cgm_df.sort_values(by='datetime')
insulin_df = insulin_df.sort_values(by='datetime')

In [439]:
#cgm_df.head()
#insulin_df.head()

In [440]:
total_rows = cgm_df['Sensor Glucose (mg/dL)'].count()
total_rows

51175

In [441]:
#cgm_df['Sensor Glucose (mg/dL)'].groupby([cgm_df.DateTime.dt.month, cgm_df.DateTime.dt.day, cgm_df.DateTime.dt.hour]).count().plot(kind="line")

In [442]:
cgm_df.dtypes

datetime                  datetime64[ns]
Sensor Glucose (mg/dL)           float64
dtype: object

In [443]:
cgm_df['datetime'].dt.time

Index
52859    12:08:54
52858    12:13:54
52857    12:18:54
52856    12:23:54
52855    12:28:54
           ...   
20359    13:02:27
20358    13:07:27
20357    13:12:27
20356    13:17:27
20355    13:22:27
Name: datetime, Length: 55343, dtype: object

In [444]:
date_df=cgm_df[['Sensor Glucose (mg/dL)']].groupby(cgm_df['datetime'].dt.date).count().reset_index()
date_df
len(date_df)

195

In [445]:
#date_df['datetime'] = pd.to_datetime(date_df['datetime'])
date_df.dtypes

datetime                  object
Sensor Glucose (mg/dL)     int64
dtype: object

In [448]:
#cond = (date_df['Sensor Glucose (mg/dL)'] == 142)
cond1 = (date_df['Sensor Glucose (mg/dL)'] > 288*0.8)
date_df= date_df.loc[cond1]
date_df
len_of_date = len(date_df)
len_of_date

165

In [508]:
#date_df.dtypes
#date_df.info()
#cgm_df.info()

In [450]:
date_df.loc['datetime'] = date_df['datetime'].astype(str)

In [481]:
def date_func(x):
    return x[0:10]
print(type(date_df['datetime']))

<class 'pandas.core.series.Series'>


In [544]:
cgm_df['datetime'] = cgm_df['datetime'].astype(str)
date_df['datetime'] = date_df['datetime'].astype(str)
date_list=date_df['datetime'].tolist()
#date_list


In [555]:
mode_df = insulin_df[insulin_df['Alarm'] == 'AUTO MODE ACTIVE PLGM OFF']
mode_df['datetime'] = mode_df['datetime'].astype(str)
mode_list = mode_df['datetime'].tolist()
mode_list

C:\Users\JOHN\anaconda3\envs\coursera\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


['2017-08-09 08:07:13', '2017-11-04 17:36:46']

In [575]:
#datetime_object = datetime.strptime('Jun 1 2005  1:33PM', '%b %d %Y %I:%M%p)
#datetime_object = datetime.strptime('2017-08-09 08:07:13', '%Y-%b-%d  )
from datetime import datetime
from datetime import timedelta          
c = datetime.fromisoformat('2017-08-09 08:07:13')
str(datetime.fromisoformat('2017-08-09 08:07:13') + timedelta(minutes=5))

'2017-08-09 08:12:13'

In [698]:
M_cgm180 = 0
A_cgm180 = 0
M_cgm250 = 0
A_cgm250 = 0
M_cgm70_180 = 0
A_cgm70_180 = 0
M_cgm70_150 = 0
A_cgm70_150 = 0
M_cgm70 = 0
A_cgm70 = 0
M_cgm50 = 0
A_cgm50 = 0
mode = 0  #0 = Manual, 1 = Auto 
# and date[11:] >= '06:00:00' and date[11:] < '24:00:00':       

In [699]:
#overnight (midnight to 6 am)
for date, glucose in cgm_df.itertuples(index=False):
    if mode ==0 and date[0:10] in date_list and date[11:] >= '00:00:00' and date[11:] < '06:00:00':
        #print(date,date[11:],mode)
        if glucose > 180:
            M_cgm180 = M_cgm180 + 1
        if glucose > 250:
            M_cgm250 = M_cgm250 + 1
        if glucose >= 70 and glucose <= 180:
            M_cgm70_180 = M_cgm70_180 + 1
        if glucose >= 70 and glucose <= 150:
            M_cgm70_150 = M_cgm70_150 + 1
        if glucose < 70:
            M_cgm70 = M_cgm70 + 1
        if glucose < 50:
            M_cgm50 = M_cgm50 + 1
    if mode ==1 and date[0:10] in date_list and date[11:] >= '00:00:00' and date[11:] < '06:00:00':
        #print(date,date[11:],mode)
        if glucose > 180:
            A_cgm180 = A_cgm180 + 1
        if glucose > 250:
            A_cgm250 = A_cgm250 + 1
        if glucose >= 70 and glucose <= 180:
            A_cgm70_180 = A_cgm70_180 + 1
        if glucose >= 70 and glucose <= 150:
            A_cgm70_150 = A_cgm70_150 + 1
        if glucose < 70:
            A_cgm70 = A_cgm70 + 1
        if glucose < 50:
            A_cgm50 = A_cgm50 + 1
    #print(date, glucose,mode)
    for d in mode_list:
        if date < d and d < str(datetime.fromisoformat(date) + timedelta(minutes=5)):
            if mode == 0: mode = mode + 1
            elif mode == 1: mode = mode - 1
            #print(date, d, mode)
 

In [700]:
print('      cgm > 180 :', M_cgm180/len_of_date/288*100)        
print('      cgm > 250 :', M_cgm250/len_of_date/288*100)
print('70 <= cgm <= 180:', M_cgm70_180/len_of_date/288*100)
print('70 <= cgm <= 150:', M_cgm70_150/len_of_date/288*100)
print('      cgm < 70  :', M_cgm70/len_of_date/288*100)        
print('      cgm < 50  :', M_cgm50/len_of_date/288*100)
print('      cgm > 180 :', A_cgm180/len_of_date/288*100)        
print('      cgm > 250 :', A_cgm250/len_of_date/288*100)
print('70 <= cgm <= 180:', A_cgm70_180/len_of_date/288*100)
print('70 <= cgm <= 150:', A_cgm70_150/len_of_date/288*100)
print('      cgm < 70  :', A_cgm70/len_of_date/288*100)        
print('      cgm < 50  :', A_cgm50/len_of_date/288*100)

o_list1 = [M_cgm180/len_of_date/288*100, A_cgm180/len_of_date/288*100]
o_list2 = [M_cgm250/len_of_date/288*100, A_cgm250/len_of_date/288*100]
o_list3 = [M_cgm70_180/len_of_date/288*100, A_cgm70_180/len_of_date/288*100]
o_list4 = [M_cgm70_150/len_of_date/288*100, A_cgm70_150/len_of_date/288*100]
o_list5 = [M_cgm70/len_of_date/288*100, A_cgm70/len_of_date/288*100]
o_list6 = [M_cgm50/len_of_date/288*100, A_cgm50/len_of_date/288*100]


      cgm > 180 : 1.9191919191919191
      cgm > 250 : 0.2546296296296296
70 <= cgm <= 180: 10.801767676767676
70 <= cgm <= 150: 8.973063973063972
      cgm < 70  : 0.3388047138047138
      cgm < 50  : 0.054713804713804715
      cgm > 180 : 1.0101010101010102
      cgm > 250 : 0.19570707070707072
70 <= cgm <= 180: 9.688552188552189
70 <= cgm <= 150: 8.674242424242424
      cgm < 70  : 0.2904040404040404
      cgm < 50  : 0.01893939393939394


In [701]:
M_cgm180 = 0
A_cgm180 = 0
M_cgm250 = 0
A_cgm250 = 0
M_cgm70_180 = 0
A_cgm70_180 = 0
M_cgm70_150 = 0
A_cgm70_150 = 0
M_cgm70 = 0
A_cgm70 = 0
M_cgm50 = 0
A_cgm50 = 0
mode = 0  #0 = Manual, 1 = Auto 

In [702]:
#daytime (6 am to midnight)
for date, glucose in cgm_df.itertuples(index=False):
    if mode ==0 and date[0:10] in date_list and date[11:] >= '06:00:00' and date[11:] < '24:00:00': 
        #print(date,date[11:],mode)
        if glucose > 180:
            M_cgm180 = M_cgm180 + 1
        if glucose > 250:
            M_cgm250 = M_cgm250 + 1
        if glucose >= 70 and glucose <= 180:
            M_cgm70_180 = M_cgm70_180 + 1
        if glucose >= 70 and glucose <= 150:
            M_cgm70_150 = M_cgm70_150 + 1
        if glucose < 70:
            M_cgm70 = M_cgm70 + 1
        if glucose < 50:
            M_cgm50 = M_cgm50 + 1
    if mode ==1 and date[0:10] in date_list and date[11:] >= '06:00:00' and date[11:] < '24:00:00':
        #print(date,date[11:],mode)
        if glucose > 180:
            A_cgm180 = A_cgm180 + 1
        if glucose > 250:
            A_cgm250 = A_cgm250 + 1
        if glucose >= 70 and glucose <= 180:
            A_cgm70_180 = A_cgm70_180 + 1
        if glucose >= 70 and glucose <= 150:
            A_cgm70_150 = A_cgm70_150 + 1
        if glucose < 70:
            A_cgm70 = A_cgm70 + 1
        if glucose < 50:
            A_cgm50 = A_cgm50 + 1
    #print(date, glucose,mode)
    for d in mode_list:
        if date < d and d < str(datetime.fromisoformat(date) + timedelta(minutes=5)):
            if mode == 0: mode = mode + 1
            elif mode == 1: mode = mode - 1
            #print(date, d, mode)

In [703]:
print('      cgm > 180 :', M_cgm180/len_of_date/288*100)        
print('      cgm > 250 :', M_cgm250/len_of_date/288*100)
print('70 <= cgm <= 180:', M_cgm70_180/len_of_date/288*100)
print('70 <= cgm <= 150:', M_cgm70_150/len_of_date/288*100)
print('      cgm < 70  :', M_cgm70/len_of_date/288*100)        
print('      cgm < 50  :', M_cgm50/len_of_date/288*100)
print('      cgm > 180 :', A_cgm180/len_of_date/288*100)        
print('      cgm > 250 :', A_cgm250/len_of_date/288*100)
print('70 <= cgm <= 180:', A_cgm70_180/len_of_date/288*100)
print('70 <= cgm <= 150:', A_cgm70_150/len_of_date/288*100)
print('      cgm < 70  :', A_cgm70/len_of_date/288*100)        
print('      cgm < 50  :', A_cgm50/len_of_date/288*100)
d_list1 = [M_cgm180/len_of_date/288*100, A_cgm180/len_of_date/288*100]
d_list2 = [M_cgm250/len_of_date/288*100, A_cgm250/len_of_date/288*100]
d_list3 = [M_cgm70_180/len_of_date/288*100, A_cgm70_180/len_of_date/288*100]
d_list4 = [M_cgm70_150/len_of_date/288*100, A_cgm70_150/len_of_date/288*100]
d_list5 = [M_cgm70/len_of_date/288*100, A_cgm70/len_of_date/288*100]
d_list6 = [M_cgm50/len_of_date/288*100, A_cgm50/len_of_date/288*100]


      cgm > 180 : 10.942760942760943
      cgm > 250 : 2.582070707070707
70 <= cgm <= 180: 25.547138047138045
70 <= cgm <= 150: 19.38131313131313
      cgm < 70  : 2.1001683501683504
      cgm < 50  : 0.5513468013468014
      cgm > 180 : 10.170454545454545
      cgm > 250 : 2.760942760942761
70 <= cgm <= 180: 21.79503367003367
70 <= cgm <= 150: 16.142676767676768
      cgm < 70  : 1.466750841750842
      cgm < 50  : 0.22516835016835016


In [704]:
M_cgm180 = 0
A_cgm180 = 0
M_cgm250 = 0
A_cgm250 = 0
M_cgm70_180 = 0
A_cgm70_180 = 0
M_cgm70_150 = 0
A_cgm70_150 = 0
M_cgm70 = 0
A_cgm70 = 0
M_cgm50 = 0
A_cgm50 = 0
mode = 0  #0 = Manual, 1 = Auto 

In [705]:
#whole day (12 am to 12 am).
for date, glucose in cgm_df.itertuples(index=False):
    if mode ==0 and date[0:10] in date_list:
        if glucose > 180:
            M_cgm180 = M_cgm180 + 1
        if glucose > 250:
            M_cgm250 = M_cgm250 + 1
        if glucose >= 70 and glucose <= 180:
            M_cgm70_180 = M_cgm70_180 + 1
        if glucose >= 70 and glucose <= 150:
            M_cgm70_150 = M_cgm70_150 + 1
        if glucose < 70:
            M_cgm70 = M_cgm70 + 1
        if glucose < 50:
            M_cgm50 = M_cgm50 + 1
    if mode ==1 and date[0:10] in date_list:
        if glucose > 180:
            A_cgm180 = A_cgm180 + 1
        if glucose > 250:
            A_cgm250 = A_cgm250 + 1
        if glucose >= 70 and glucose <= 180:
            A_cgm70_180 = A_cgm70_180 + 1
        if glucose >= 70 and glucose <= 150:
            A_cgm70_150 = A_cgm70_150 + 1
        if glucose < 70:
            A_cgm70 = A_cgm70 + 1
        if glucose < 50:
            A_cgm50 = A_cgm50 + 1
    #print(date, glucose,mode)
    for d in mode_list:
        if date < d and d < str(datetime.fromisoformat(date) + timedelta(minutes=5)):
            if mode == 0: mode = mode + 1
            elif mode == 1: mode = mode - 1
            #print(date, d, mode)
  

In [706]:
print('      cgm > 180 :', M_cgm180/len_of_date/288*100)        
print('      cgm > 250 :', M_cgm250/len_of_date/288*100)
print('70 <= cgm <= 180:', M_cgm70_180/len_of_date/288*100)
print('70 <= cgm <= 150:', M_cgm70_150/len_of_date/288*100)
print('      cgm < 70  :', M_cgm70/len_of_date/288*100)        
print('      cgm < 50  :', M_cgm50/len_of_date/288*100)
print('      cgm > 180 :', A_cgm180/len_of_date/288*100)        
print('      cgm > 250 :', A_cgm250/len_of_date/288*100)
print('70 <= cgm <= 180:', A_cgm70_180/len_of_date/288*100)
print('70 <= cgm <= 150:', A_cgm70_150/len_of_date/288*100)
print('      cgm < 70  :', A_cgm70/len_of_date/288*100)        
print('      cgm < 50  :', A_cgm50/len_of_date/288*100)
a_list1 = [M_cgm180/len_of_date/288*100, A_cgm180/len_of_date/288*100]
a_list2 = [M_cgm250/len_of_date/288*100, A_cgm250/len_of_date/288*100]
a_list3 = [M_cgm70_180/len_of_date/288*100, A_cgm70_180/len_of_date/288*100]
a_list4 = [M_cgm70_150/len_of_date/288*100, A_cgm70_150/len_of_date/288*100]
a_list5 = [M_cgm70/len_of_date/288*100, A_cgm70/len_of_date/288*100]
a_list6 = [M_cgm50/len_of_date/288*100, A_cgm50/len_of_date/288*100]

      cgm > 180 : 12.86195286195286
      cgm > 250 : 2.8367003367003365
70 <= cgm <= 180: 36.34890572390572
70 <= cgm <= 150: 28.354377104377104
      cgm < 70  : 2.438973063973064
      cgm < 50  : 0.6060606060606061
      cgm > 180 : 11.180555555555555
      cgm > 250 : 2.9566498316498318
70 <= cgm <= 180: 31.48358585858586
70 <= cgm <= 150: 24.81691919191919
      cgm < 70  : 1.7571548821548821
      cgm < 50  : 0.24410774410774408


In [708]:
results_df = pd.DataFrame(list(zip(o_list1, o_list2, o_list3, o_list4, o_list5, o_list6, d_list1, d_list2, d_list3, d_list4, d_list5, d_list6, a_list1, a_list2, a_list3, a_list4, a_list5, a_list6)))
results_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,1.919192,0.254630,10.801768,8.973064,0.338805,0.054714,10.942761,2.582071,25.547138,19.381313,2.100168,0.551347,12.861953,2.83670,36.348906,28.354377,2.438973,0.606061
1,1.010101,0.195707,9.688552,8.674242,0.290404,0.018939,10.170455,2.760943,21.795034,16.142677,1.466751,0.225168,11.180556,2.95665,31.483586,24.816919,1.757155,0.244108


In [712]:
results_df.to_csv(r'Results.csv', index = False, header=False)